<a href="https://colab.research.google.com/github/samarasss/ri_lab_01/blob/master/lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install memory_profiler
import pandas as pd
import numpy as np
import nltk
import heapq
import collections
import time
%load_ext memory_profiler

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

nltk.download('stopwords')




     |████████████████████████████████| 40kB 2.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/f0/ff/63/fdbff3f1e1b76ad4eae491dd5b190902906b093e93eb86dd5a
Successfully built memory-profiler
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
url = 'https://raw.githubusercontent.com/samarasss/Recupera-o-da-Informa-o/master/dados/results.csv'
df1 = pd.read_csv(url).replace(np.nan, '', regex=True)

pattern = r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})'
tokenize = RegexpTokenizer(pattern)
stop_words = stopwords.words("portuguese")

words = []

for word in df1.text:
  tokens = [token for token in tokenize.tokenize(word.lower()) 
           if len(token)>= 3 and token not in stop_words]
  words.extend(tokens)



**1.Execute o algoritmo ilustrado na Fig. 5.8 do livro texto (pag. 157) para gerar um índice similar o mostrado na Fig. 5.4 (pag. 134)**

In [0]:
def buildIndex(words):
  indexes = {}
  n = 0
  for document in words:
    n = n + 1
    tokens = set()
    terms = [token for token in tokenize.tokenize(document)]#tokens from document
    tokens.update(terms)
    
    #remova duplicatas de tokens
    for token in tokens:
      occurrence = document.count(token)
      if token not in indexes:
        indexes[token] = []
      indexes[token].append(tuple((n,occurrence)))
  return indexes

#indexes = buildIndex(df1.text)

#valores = {'token':list(indexes.keys()), 'oc_list':list(indexes.values())}
#df = pd.DataFrame(valores)

In [0]:
indice_invertido =  buildIndex(df1.text)

indice_invertido_table = pd.DataFrame()
indice_invertido_table["Word"] = indice_invertido.keys()
indice_invertido_table["DocId:Occurrence"] = indice_invertido.values()

**Guarde o índice em disco em formato csv.**



In [0]:
indice_invertido_table.to_csv("indice_invertido.csv", encoding="utf-8", index=False)


**2.Implemente as abordagens de processamento de consulta documento-por-vez e termo-por-vez (Fig. 5.16 e 5.18).**

In [0]:
def documentAtTimeRetrieval(query, indice_invertido, k):
    
    lista_invertida = []
    rank = []
    for word in query.split(" "):
        if word in indice_invertido.keys():
            lista_invertida.append(indice_invertido[word])
    for document in range(1, len(words)+1):
        sd = 0
        for index in lista_invertida:
            for element in index:
                if (element[0] == document):
                    sd += element[1]
                    break
        if (sd != 0):
          heapq.heappush(rank, (sd, document))
    return heapq.nlargest(k, rank)
      

In [0]:
def termAtTimeRetrieval(query, indice_invertido, k):
  
    a = {}
    lista_invertida = []
    rank = []
    for word in query.split(" "):
        if word in indice_invertido.keys():
            lista_invertida.append(indice_invertido[word])
    for index in lista_invertida:
        for post in index:
            d = post[0]
            freq = post[1]
            if (d in a.keys()):
                a[d] += freq
            else:
                a[d] = freq
    for (d, ad) in a.items():
        sd = ad
        heapq.heappush(rank, (sd, d))
    return heapq.nlargest(k, rank)

**1.Defina 5 consultas de um termo somente.**


In [0]:
 queries = ["conforme", "processo", "revolução", "Guerra", "responsáveis"]


**2.Execute as 5 consultas em cada algoritmo retornando os top-10 documentos (parâmetro k do algoritmo).**


In [9]:
resultDocuments = []
resultTerms = []
timeResultDocuments = []
timeResultTerms = []
k = 10

for query in queries:
  init_document = time.time()
  resultDocuments.append(documentAtTimeRetrieval(query, indice_invertido, k))
  end_document = time.time()
  timeResultDocuments.append(end_document - init_document)
  
  init_term = time.time()
  resultTerms.append(termAtTimeRetrieval(query, indice_invertido, k))
  end_term = time.time()
  timeResultTerms.append(end_term - init_term)

terms_table = pd.DataFrame()
terms_table['query'] = queries
terms_table['document_at_a_time'] = resultDocuments
terms_table['term_at_a_time'] = resultTerms
terms_table['compare'] = terms_table.document_at_a_time == terms_table.term_at_a_time
terms_table

,query,document_at_a_time,term_at_a_time,compare
0,conforme,"[(1, 249), (1, 235), (1, 225), (1, 193), (1, 1...","[(1, 249), (1, 235), (1, 225), (1, 193), (1, 1...",True
1,processo,"[(8, 63), (7, 40), (5, 227), (5, 202), (5, 161...","[(8, 63), (7, 40), (5, 227), (5, 202), (5, 161...",True
2,revolução,"[(2, 86), (2, 25), (2, 16), (1, 208), (1, 190)...","[(2, 86), (2, 25), (2, 16), (1, 208), (1, 190)...",True
3,Guerra,"[(2, 223), (2, 111), (2, 4), (1, 248), (1, 244...","[(2, 223), (2, 111), (2, 4), (1, 248), (1, 244...",True
4,responsáveis,"[(2, 16), (1, 222), (1, 220), (1, 207), (1, 18...","[(2, 16), (1, 222), (1, 220), (1, 207), (1, 18...",True


In [10]:
print('Documento/uso de memória:')
%memit documentAtTimeRetrieval(queries[0], indice_invertido, k)

print('Termo / uso de memória:')
%memit termAtTimeRetrieval(queries[0], indice_invertido, k)

Documento/uso de memória:
peak memory: 204.74 MiB, increment: 0.12 MiB
Termo / uso de memória:
peak memory: 204.74 MiB, increment: 0.00 MiB


**3.Dê evidências de que sua implementação está correta.**


**Compare os tempos médios de execução e uso de memória de cada algoritmo.**


In [11]:
tempo_df = pd.DataFrame()
tempo_df['tempo_médio_document_at_a_time'] = timeResultDocuments
tempo_df['tempo_médio_term_at_a_time'] = timeResultTerms
tempo_df

,tempo_médio_document_at_a_time,tempo_médio_term_at_a_time
0,0.122530,0.000040
1,0.440433,0.000107
2,0.113475,0.000040
3,0.112314,0.000038
4,0.157252,0.000047


**3.Implemente uma das versões de consulta conjuntiva (AND) (Fig. 5.20 ou 5.21).**
        1.Defina 5 consultas conjuntivas (AND).
        2.Execute as 5 consultas em cada algoritmo retornando os top-10    documentos (parâmetro k do algoritmo).
        3.Dê evidências de que sua implementação está correta*

In [0]:
def conj_query(query, indice_invertido, k):
  
  query_indexes = []
  rank = []
  
  for word in query.split(" "):
    if word in indice_invertido.keys():
        query_indexes.append(indice_invertido[word])
  
  all_query = [item for sublist in query_indexes for item in sublist]
  all_query.sort()
  
  for i in range(len(all_query)):
    doc_score = 0
    d = all_query.pop()
    for index in all_query:
      if index[0] == d[0]:
        doc_score += index[1] + d[1]
    if doc_score != 0:
      heapq.heappush(rank, (doc_score, d[0]))
        
  return heapq.nlargest(k, rank)



In [0]:
queries_conj = ["porta legal", "porta questão", "proibição política", "Carta Aeronáutica", "princípio golpistas"]
k  = 10
results_conj = []

for query in queries_conj:
  score_conj = conj_query(query, indice_invertido, k)
  results_conj.append(score_conj)

In [14]:
result_df = pd.DataFrame()
result_df['Consulta'] = queries_conj
result_df['Resultado'] = results_conj
result_df.index+=1
result_df

,Consulta,Resultado
1,porta legal,"[(4, 1), (2, 125), (2, 124)]"
2,porta questão,"[(9, 25), (8, 166), (7, 230), (4, 1), (3, 208)..."
3,proibição política,"[(3, 90), (2, 161), (2, 1)]"
4,Carta Aeronáutica,"[(2, 1)]"
5,princípio golpistas,"[(4, 1)]"
